# 🍎 Health Calculator Agent Tutorial 🍏

Welcome to the **Health Calculator Agent** tutorial, where we'll showcase how to:

- **Initialize** a project and use the Azure AI Foundry ecosystem
- **Create an Agent** with **Code Interpreter** capabilities
- **Perform BMI calculations** and **analyze nutritional data** with sample CSV files
- **Generate** basic health insights and disclaimers
    >    **Ensure you have completed the [1-basics.ipynb](./1-basics.ipynb) notebook before starting this one.**

## Let's Dive In

We'll walk step-by-step, similar to our **Fun & Fit** sample, but with a focus on using **Code Interpreter** for numeric calculations and data analysis. Let's begin!

<img src="seq-diagrams/2-code-interpreter.png" alt="Code Interpreter Sequence Diagram" style="width:700px;"/>

# 🔐 Authentication Setup

Before running the next cell, make sure you're authenticated with Azure CLI. Run this command in your terminal:

```az login --use-device-code```

This will provide you with a device code and URL to authenticate in your browser, which is useful for:

- Remote development environments
- Systems without a default browser
- Corporate environments with strict security policies

After successful authentication, you can proceed with the notebook cells below.

# 1. Initial Setup

We'll start by importing libraries, loading environment variables, and initializing an AIProjectClient. We'll also create a sample CSV for demonstration.

In [ ]:
#pragma warning disable OPENAI001

#r "nuget: Azure.Identity, 1.18.0-beta.2"
#r "nuget: Azure.AI.Projects, 1.2.0-beta.5"
#r "nuget: dotenv.net"

using System;
using System.Text;
using System.Globalization;
using System.IO;
using System.ClientModel.Primitives;
using System.Reflection;
using Azure.Identity;
using Azure.AI.Projects;
using Azure.AI.Projects.OpenAI;
using OpenAI.Responses;
using OpenAI.Files;
using dotenv.net;  

DotEnv.Load(new DotEnvOptions(envFilePaths: new[] { Path.Combine(".","..", ".env") })); 

In [ ]:
var tenantId = Environment.GetEnvironmentVariable("TENANT_ID");
var projectEndpoint = Environment.GetEnvironmentVariable("AI_FOUNDRY_PROJECT_ENDPOINT");
var modelDeployment = Environment.GetEnvironmentVariable("MODEL_DEPLOYMENT_NAME");
Console.WriteLine($"🔑 Using Tenant ID: {tenantId}");

AIProjectClient projectClient;

try
{
    var credential = new DefaultAzureCredential(new DefaultAzureCredentialOptions
    {
        TenantId = tenantId
    });

    projectClient = new AIProjectClient(new Uri(projectEndpoint), credential);
    Console.WriteLine("✅ Successfully initialized AIProjectClient");
}
catch (Exception ex)
{
    Console.WriteLine($"❌ Error initializing AIProjectClient: {ex.Message}");
    throw;
}

In [ ]:
string? CreateSampleData()
{
    try
    {
        var start = new DateTime(2024, 1, 1);
        int periods = 7;

        int[] calories = { 2100, 1950, 2300, 2050, 1900, 2200, 2150 };
        int[] proteinG = { 80, 75, 85, 78, 72, 82, 79 };
        int[] carbsG   = { 250, 230, 270, 245, 225, 260, 255 };
        int[] fatG     = { 70, 65, 75, 68, 63, 73, 71 };
        int[] fiberG   = { 25, 22, 28, 24, 21, 26, 23 };

        var filename = Environment.GetEnvironmentVariable("NUTRITION_DATA_FILENAME")
                       ?? "nutrition_data.csv";

        var sb = new StringBuilder();
        sb.AppendLine("Date,Calories,Protein_g,Carbs_g,Fat_g,Fiber_g");

        for (int i = 0; i < periods; i++)
        {
            var date = start.AddDays(i).ToString("yyyy-MM-dd", CultureInfo.InvariantCulture);
            sb.AppendLine($"{date},{calories[i]},{proteinG[i]},{carbsG[i]},{fatG[i]},{fiberG[i]}");
        }

        File.WriteAllText(filename, sb.ToString(), Encoding.UTF8);
        Console.WriteLine($"📄 Created sample data file: {filename}");
        return filename;
    }
    catch (Exception e)
    {
        Console.WriteLine($"❌ Error creating sample data: {e}");
        return null;
    }
}

var sampleFile = CreateSampleData();

# 2. Create Health Calculator Agent 👩‍💻

We'll upload our sample CSV and then create an agent with **Code Interpreter** enabled. This agent can read the file, run Python code, and return results and visualizations.

In [ ]:
#pragma warning disable OPENAI001

async Task<(AgentVersion, OpenAIFile)> CreateHealthCalculator(string filePath)
{
    try
    {
    Console.WriteLine($"📤Uploading file: {filePath}");
    var uploadedFile = await projectClient.OpenAI.Files.UploadFileAsync(filePath, FileUploadPurpose.Assistants);
    Console.WriteLine($"✅ Uploaded CSV file, ID: {uploadedFile.Value.Id}");

    var agentDefinition = new PromptAgentDefinition(modelDeployment)
        {
            Instructions = @"
            You are a health calculator agent that can:
            1. Calculate and interpret BMI using the formula: BMI = weight(kg) / height(m)²
            2. Analyze provided nutrition data from CSV files that are attached to messages
            3. Generate charts and plots for data visualization
            4. Always include disclaimers that you are not a medical professional and cannot provide medical advice
            5. Use code interpreter to perform calculations and create visualizations
            
            When working with uploaded files:
            - Files will be attached to individual messages for you to analyze
            - You can access them directly using pandas or other Python libraries in code interpreter
            - Look for CSV files containing nutrition data with columns: Date, Calories, Protein_g, Carbs_g, Fat_g, Fiber_g
            
            When providing health information, always remind users that:
            - This is for educational purposes only
            - They should consult healthcare professionals for medical advice
            - Individual health needs vary significantly
            ",
            Tools =
            {
                ResponseTool.CreateCodeInterpreterTool(
                    new CodeInterpreterToolContainer(
                        CodeInterpreterToolContainerConfiguration.CreateAutomaticContainerConfiguration(fileIds:[uploadedFile.Value.Id])
                    )
                )
            }
        };
        AgentVersion agentVersion = await projectClient.Agents.CreateAgentVersionAsync(
            agentName:"health-calculator-agent", 
            options: new(agentDefinition)
        );
        Console.WriteLine($"🎉 Created health calculator agent, ID: {agentVersion.Id}");
        Console.WriteLine($"📄 File {uploadedFile.Value.Id} will be attached to individual messages when needed");
        return (agentVersion, uploadedFile);  
    }
    catch (Exception ex)
    {
        Console.WriteLine($"❌ Error creating health calculator agent: {ex.Message}");
        return (null, null);
    }
}

In [ ]:
// Recreate the agent with the working approach

Console.WriteLine("🔄 Recreating agent with message-level file attachment approach...");
(AgentVersion healthAgent, OpenAIFile uploadedFile) = (null, null);
if (sampleFile != null)
{
    (healthAgent, uploadedFile) = await CreateHealthCalculator(sampleFile);
}

# 3. BMI Calculation with Code Interpreter

We'll create a thread for BMI calculations. We'll feed in the user's height/weight, and ask the agent to show how it calculates BMI, interpret the result, and always disclaim professional advice.

In [ ]:
#pragma warning disable OPENAI001

async Task<(string, ResponseResult)> CalculateBmiWithAgent(AgentVersion agent, double heightInches, double weightPounds)
{
    try
    {
           var conversation = await projectClient.OpenAI.Conversations.CreateProjectConversationAsync();
           Console.WriteLine($"📝 Created conversation for BMI calculation, ID: {conversation.Value.Id}");

           var userText = @$"
            Calculate BMI for:
            Height: {heightInches} inches
            Weight: {weightPounds} pounds

            Please:
            1. Show the calculation step by step
            2. Convert units to metric if needed
            3. Interpret the BMI result (underweight, normal, overweight, obese)
            4. Include appropriate health disclaimers
            5. Use Python code to perform the calculations";

            var responsesClient = projectClient.OpenAI.GetProjectResponsesClientForAgent(
                defaultAgent: healthAgent,
                defaultConversationId: conversation.Value.Id);
            var responseResult = await responsesClient.CreateResponseAsync(userText);
            Console.WriteLine($"🤖 BMI run finished with status: {responseResult.Value.Status}");        
            return (conversation.Value.Id, responseResult.Value);
    }
    catch (Exception ex)
    {
        Console.WriteLine($"❌ Error during BMI calculation: {ex.Message}");
        
        return (null, null);
    }
}



In [ ]:
#pragma warning disable OPENAI001
string bmiConversationId = null;
if(healthAgent != null)
{
    double heightInches = 70; // Example height
    double weightPounds = 180; // Example weight

    (bmiConversationId, ResponseResult bmiResponse) = await CalculateBmiWithAgent(healthAgent, heightInches, weightPounds);
}

# 4. Nutrition Analysis

We'll create another thread where the user can ask the agent to analyze the nutrition data CSV we uploaded. The agent can read the file, compute macros, produce charts, and disclaim that it's not offering personalized medical advice.

In [ ]:
#pragma warning disable OPENAI001

async Task<(string, ResponseResult)> AnalyzeNutritionData(AgentVersion agent)
{
        var conversation = await projectClient.OpenAI.Conversations.CreateProjectConversationAsync();
        Console.WriteLine($"📝 Created conversation for Nutrition Analysis, ID: {conversation.Value.Id}");

    var userText = @$"
            Please analyze the attached nutrition CSV file.
            
            Please:
            1. Load and examine the data structure 
            2. Compute average daily macros (calories, protein, carbs, fat, fiber)
            3. Create visualizations to show trends over the week
            4. Identify any interesting patterns or insights
            5. Provide general health commentary with appropriate disclaimers
            6. Use Python code for all calculations and visualizations

            The CSV file contains daily nutrition data with columns: Date, Calories, Protein_g, Carbs_g, Fat_g, Fiber_g
        ";

        try
        {
            var responsesClient = projectClient.OpenAI.GetProjectResponsesClientForAgent(
                defaultAgent: healthAgent,
                defaultConversationId: conversation.Value.Id);
            var responseResult = await responsesClient.CreateResponseAsync(userText);
            Console.WriteLine($"🤖 Nutrition Analysis run finished with status: {responseResult.Value.Status}");        
            return (conversation.Value.Id, responseResult.Value);
        }
        catch(Exception ex)
        {
                Console.WriteLine($"❌ Error during Nutrition Analysis: {ex.Message}");
                return (null, null);
        }

}

sampleFile = CreateSampleData();

(healthAgent, uploadedFile) = await CreateHealthCalculator(sampleFile);

string nutritionAnalysisConversationId = null;
ResponseResult nutritionAnalysisResponse = null;
(nutritionAnalysisConversationId, nutritionAnalysisResponse) = await AnalyzeNutritionData(healthAgent);


# 5. Viewing Results & Visualizations 📊

The agent may produce text insights, disclaimers, and even images with charts. Let's fetch them from our threads!

In [ ]:
#pragma warning disable OPENAI001

async Task ViewAgentResponse(string conversationId)
{
    try
    {
        Console.WriteLine($"\n🔎 Agent Responses from Thread: {conversationId}");
        Console.WriteLine(new string('=', 60));
        var itemCount = 0;
        await foreach (AgentResponseItem item in projectClient.OpenAI.Conversations.GetProjectConversationItemsAsync(
                        conversationId, order:"asc"
                       ))
        {
            itemCount++;
            var responseResultItem = item.AsResponseResultItem();
            
            if(responseResultItem is MessageResponseItem)
            {
                var messageResponseItem = responseResultItem as MessageResponseItem;
                var role = messageResponseItem.Role.ToString();
                var roleEmoji = role == "User" ? "👤" : "🤖";
                Console.WriteLine($"{itemCount}. {roleEmoji} {role}:");
                Console.WriteLine(new string('-', 40));
                var content = messageResponseItem.Content?[0];
                var contentText = content?.Text;
                if(contentText != null && contentText.Length > 0)
                {
                    if(contentText.Length > 1000)
                    {
                        Console.WriteLine($"{contentText.Substring(0,1000)}... [truncated]");
                    } 
                    else
                    {
                        Console.WriteLine(contentText);
                    }
                }
                else
                {
                    Console.WriteLine("No content available.");
                }
                if(content.OutputTextAnnotations != null && content.OutputTextAnnotations.Count > 0)
                {
                    foreach(var annotation in content.OutputTextAnnotations)
                    {
                        if(annotation is ContainerFileCitationMessageAnnotation)
                        {
                            var fileAnnotation = annotation as ContainerFileCitationMessageAnnotation;
                            var fileId = fileAnnotation.FileId;
                            var filename = fileAnnotation.Filename;
                            Console.WriteLine($"🖼️ Image generated with file ID: {fileId}, Filename: {filename}");
                            var containerFile = await projectClient.OpenAI.GetContainerClient().DownloadContainerFileAsync(fileAnnotation.ContainerId, fileId);
                            var containerFileValue = containerFile.Value;
                            var localFilePath = Path.Combine(".",fileAnnotation.Filename);
                            Console.WriteLine($"💾 Saving file to: {localFilePath}");
                            await File.WriteAllBytesAsync(localFilePath,containerFileValue.ToArray());
                        }
                    }
                }
            }
        }
    }
    catch (Exception ex)
    {
        Console.WriteLine($"❌ Error viewing agent response: {ex.Message} {Environment.NewLine} {ex.StackTrace}");
    }
}

if(bmiConversationId != null)
{
    Console.WriteLine("".PadRight(80,'='));
    Console.WriteLine("🧮 BMI CALCULATION RESULTS");
    Console.WriteLine("".PadRight(80,'='));
    await ViewAgentResponse(bmiConversationId);
}
if(nutritionAnalysisConversationId != null)
{
    Console.WriteLine("".PadRight(80,'='));
    Console.WriteLine("🍎 NUTRITION ANALYSIS RESULTS");
    Console.WriteLine("".PadRight(80,'='));
    await ViewAgentResponse(nutritionAnalysisConversationId);
}

# 6. Cleanup & Best Practices

We can remove our agent and sample data if desired. In production, you might keep them for repeated usage.

### Best Practices in a Nutshell

1. **Data Handling** – Validate input data, handle missing values, properly manage file attachments.
2. **Calculations** – Provide formula steps, disclaimers, limit scope to general wellness, remind user you're not a doctor.
3. **Visualizations** – Use clear labeling and disclaimers that charts are for educational demonstrations.
4. **Security** – Monitor usage, limit access to code interpreter if dealing with proprietary data.

In [ ]:
async Task CleanupAll()
{
    try
    {
        await projectClient.OpenAI.Files.DeleteFileAsync(uploadedFile.Id);
        Console.WriteLine($"🗑️ Deleted uploaded file: {uploadedFile.Id}");
    }
    catch(Exception ex)
    {
        Console.WriteLine($"❌ Error deleting uploaded file: {ex.Message}");
    }
    
    try
    {
        await projectClient.Agents.DeleteAgentAsync(healthAgent.Name);
        Console.WriteLine($"🗑️ Deleted agent version: {healthAgent.Id}");
    }
    catch(Exception ex)
    {
        Console.WriteLine($"❌ Error deleting agent version: {ex.Message}");
    }
    
    if(File.Exists(sampleFile))
    {
        File.Delete(sampleFile);
        Console.WriteLine($"🗑️ Deleted sample data file: {sampleFile}");
    }

    Console.WriteLine("✅ Cleanup completed.");
}

// Option to run cleanup - you can comment this out if you want to keep the resources
Console.WriteLine("🧹 Running cleanup to remove created resources...");
Console.WriteLine("💡 Comment out the cleanup_all() call if you want to keep the agent and files for further testing.");
await CleanupAll();

# Congratulations! 🎉

You've successfully completed the Health Calculator Agent with Code Interpreter tutorial! Here's what you accomplished:

## ✅ What We Built

### 🤖 Health Calculator Agent

- Created an AI agent with Code Interpreter capabilities
- Enabled the agent to write and execute Python code in a secure sandbox
- Configured health-focused instructions with appropriate disclaimers

## 📊 Key Features Demonstrated

1. 📁 File Upload & Management

    - Uploaded CSV nutrition data to the Azure AI service
    - Learned correct file upload methods and purposes
    - Fixed CSV file access issues by using message-level file attachments

2. 🧮 BMI Calculations

    - Agent performed step-by-step BMI calculations using Python code
    - Converted between imperial and metric units
    - Interpreted results with health categories
    - Provided responsible AI disclaimers

3. 📈 Nutrition Data Analysis

    - Agent loaded and analyzed CSV data programmatically
    - Computed statistical averages for macronutrients
    - Generated data visualizations and trend analysis
    - Provided insights with appropriate health disclaimers

4. 🔧 Resource Management

    - Properly cleaned up agents, files, and local resources
    - Demonstrated best practices for resource lifecycle management


## 🎯 Key Concepts Learned

- **Code Interpreter Tool**: Enables agents to write and run Python code
- **File Integration**: How to upload and attach files to agent conversations
- **Secure Execution**: Code runs in a sandboxed environment
- **Data Analysis**: Agents can perform statistical analysis and create visualizations
- **Responsible AI**: Always include appropriate disclaimers for health-related content
- **Troubleshooting**: How to debug and fix file access issues

## 🚀 What's Next?

Continue your Azure AI Agent Service journey with these advanced topics:

- [**3-file-search.ipynb**](./3-file-search.ipynb) - Agents that can search through documents and knowledge bases
- [**4-bing_grounding.ipynb**](./4-bing_grounding.ipynb) - Agents with real-time web search capabilities
- [**5-agents-aisearch.ipynb**](./5-agents-aisearch.ipynb) - Integration with Azure AI Search for enterprise knowledge
- [**6-agents-az-functions.ipynb**](./6-agents-az-functions.ipynb) - Agents that can trigger Azure Functions and workflows

Ready to explore more advanced agent capabilities? Let's continue! 🚀

Happy (healthy) coding! 💪
